# [Advent of Code 2022 Day ?]()

In [1]:
from __future__ import annotations
import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2
run_doctest_for = lambda func: doctest.run_docstring_examples(func, globals())

## Test Cases

In [2]:
PART_ONE_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 26,
    },
    "input": {
        "input1": 5108096,
    },
}
PART_ONE_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}
PART_ONE_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}

In [3]:
PART_TWO_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 56000011,
    },
    "input": {
        "input1": 10553942650264,
    },
}
PART_TWO_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}
PART_TWO_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}

## Input Parsing

In [4]:
class Model(BaseModel):
    data: Any

def parse_input_from_filename(filename: str) -> Context:
    lines = list(yield_line(filename))

    ctx = Context()
    ctx.input = []

    input_lines = ctx.input

    for idx, line in enumerate(lines):
        s1, s2, b1, b2 = parse(r"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)", line)
        input_lines.append((Point(int(s1), int(s2)), Point(int(b1), int(b2))))

    return ctx

In [5]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["example"].keys() | PART_TWO_CASES["example"].keys())
def test_parsing_examples(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

.                                                                                            [100%]
============================================= PASSES ==============================================
_________________________________ test_parsing_examples[example1] _________________________________
-------------------------------------- Captured stderr call ---------------------------------------
(Point(x=2, y=18), Point(x=-2, y=15))
(Point(x=9, y=16), Point(x=10, y=16))
(Point(x=13, y=2), Point(x=15, y=3))
(Point(x=12, y=14), Point(x=10, y=16))
(Point(x=10, y=20), Point(x=10, y=16))
(Point(x=14, y=17), Point(x=10, y=16))
(Point(x=8, y=7), Point(x=2, y=10))
(Point(x=2, y=0), Point(x=2, y=10))
(Point(x=0, y=11), Point(x=2, y=10))
(Point(x=20, y=14), Point(x=25, y=17))
(Point(x=17, y=20), Point(x=21, y=22))
(Point(x=16, y=7), Point(x=15, y=3))
(Point(x=14, y=3), Point(x=15, y=3))
(Point(x=20, y=1), Point(x=15, y=3))
1 passed in 0.02s


In [6]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["input"].keys() | PART_TWO_CASES["input"].keys())
def test_parsing_inputs(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

.                                                                                            [100%]
============================================= PASSES ==============================================
___________________________________ test_parsing_inputs[input1] ___________________________________
-------------------------------------- Captured stderr call ---------------------------------------
(Point(x=2288642, y=2282562), Point(x=1581951, y=2271709))
(Point(x=2215505, y=2975419), Point(x=2229474, y=3709584))
(Point(x=275497, y=3166843), Point(x=-626874, y=3143870))
(Point(x=1189444, y=2115305), Point(x=1581951, y=2271709))
(Point(x=172215, y=2327851), Point(x=-101830, y=2000000))
(Point(x=3953907, y=1957660), Point(x=2882446, y=1934422))
(Point(x=685737, y=2465261), Point(x=1581951, y=2271709))
(Point(x=1458348, y=2739442), Point(x=1581951, y=2271709))
(Point(x=3742876, y=2811554), Point(x=3133845, y=3162635))
(Point(x=437819, y=638526), Point(x=-101830, y=2000000))
(Point(x=253797

## Helper Functions

### Helper 1

In [7]:
%%ipytest -xrPvvvvv

@lru_cache(None)
def manhattan(point1: Point, point2: Point) -> int:
    x1, y1 = point1
    x2, y2 = point2
    return abs(x1 - x2) + abs(y1 - y2)

def test_helper_1() -> None:
    assert manhattan(Point(1, 1), Point(2, 2)) == 2

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


### Helper 2

In [8]:
%%ipytest -xrPvvvvv

@lru_cache(None)
def get_manhattan(point: Point, dist: int) -> set[Point]:
    result = set()
    x, y = point

    for i in range(dist + 1):
        j = dist - i
        assert i + j == dist, f"Bad math"
        result.add(Point(-i + x, j + y))
        result.add(Point(i + x, -j + y))
        result.add(Point(i + x, j + y))
        result.add(Point(-i + x, -j + y))

    return result

def test_helper_2() -> None:
    assert (t1 := get_manhattan(Point(8, 7), 1)) and len(t1) == 4
    print(get_manhattan(Point(2, 10), 9))

.                                                                                            [100%]
============================================= PASSES ==============================================
__________________________________________ test_helper_2 __________________________________________
-------------------------------------- Captured stdout call ---------------------------------------
{Point(x=6, y=15), Point(x=4, y=3), Point(x=5, y=4), Point(x=-5, y=12), Point(x=5, y=16), Point(x=-3, y=6), Point(x=9, y=8), Point(x=10, y=9), Point(x=0, y=17), Point(x=1, y=18), Point(x=-4, y=7), Point(x=-1, y=4), Point(x=6, y=5), Point(x=-4, y=13), Point(x=-1, y=16), Point(x=3, y=18), Point(x=-6, y=9), Point(x=4, y=17), Point(x=-5, y=8), Point(x=-3, y=14), Point(x=1, y=2), Point(x=10, y=11), Point(x=2, y=1), Point(x=11, y=10), Point(x=2, y=19), Point(x=-7, y=10), Point(x=7, y=6), Point(x=3, y=2), Point(x=-2, y=15), Point(x=7, y=14), Point(x=-6, y=11), Point(x=8, y=7), Point(x=0, y=3), Point(

## Main Function

In [9]:
def solve(part: int, filename: str) -> int:  # not 20000 or 11,000something or 261829, or 261839, or 1101829
    input = parse_input_from_filename(filename).input
    if part == 1:
        sensors = ImmutableSet()
        beacons = set()
        bad = set()
        manhattan_limit_by_point = {}

        x_min = INF
        x_max = -INF
        y_min = INF
        y_max = -INF

        target = 10 if filename == "example1" else 2000000
        off = 1000 if filename == "example1" else 5300000

        for sensor, beacon in input:
            x, y = sensor
            sensors.add(sensor)
            beacons.add(beacon)
            dist = manhattan(sensor, beacon)
            x_min = min(x_min, x - dist)
            x_max = max(x_max, x + dist)
            y_min = min(y_min, y - dist)
            y_max = max(y_max, y + dist)
            if manhattan(sensor, Point(x, target)) > dist:
                continue
            manhattan_limit_by_point[sensor] = dist

        for x in range(-off, off):
            attempt = Point(x, target)
            for sensor, limit in manhattan_limit_by_point.items():
                if (dist := manhattan(sensor, attempt)) <= limit and attempt not in beacons:
                    bad.add(attempt)

        return len(bad)
    if part == 2:
        sensors = ImmutableSet()
        beacons = set()
        manhattan_limit_by_point = {}

        high = 20 if filename == "example1" else 4000000

        for sensor, beacon in input:
            sensors.add(sensor)
            beacons.add(beacon)
            dist = manhattan(sensor, beacon)
            manhattan_limit_by_point[sensor] = dist

        for sensor, limit in manhattan_limit_by_point.items():
            attempts = get_manhattan(sensor, limit + 1)
            for attempt in attempts:
                i, j = attempt
                if i < 0 or j < 0 or i > high or j > high:
                    continue
                for sensor, limit in manhattan_limit_by_point.items():
                    if (dist := manhattan(sensor, attempt)) <= limit or attempt in beacons or attempt in sensors:
                        break
                else:
                    print(f"returning Point({i}, {j}) -> {i * 4000000} + {j}")
                    return i * 4000000 + j

        raise Exception(f"Should have found something...")

    else:
        raise Exception(f"Invalid part: {part}")

### Part 1

In [10]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["example"].items())
def test_part_one_examples(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["input"].items())
def test_part_one_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

..                                                                                           [100%]
============================================= PASSES ==============================================
2 passed in 301.40s (0:05:01)


### Part 2

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["example"].items())
def test_part_two_examples(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["input"].items())
def test_part_two_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

.

dadada